In [97]:
import yfinance as yf
import numpy as np
import pandas as pd
from pmdarima.arima import auto_arima
from arch import arch_model
import matplotlib.pyplot as plt
from datetime import datetime

symbol = 'NVDA'

# Fetching data
spy_data = yf.download(symbol, start="2022-03-29", end="2024-03-29")

test_size = 100
train = spy_data['Close'][:-test_size]
test = spy_data['Close'][-test_size:]

returns = train.pct_change().dropna()
returns *= 100


[*********************100%%**********************]  1 of 1 completed


C:\Users\surya\AppData\Roaming\Python\Python311\site-packages\yfinance\utils.py:788: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [98]:
# Fitting ARIMA model
arima_model = auto_arima(returns, seasonal=False, trace=True)
arima_residuals = arima_model.resid()
arima_model.order

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2444.936, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2333.803, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2173.761, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2442.939, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2175.731, Time=0.20 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2175.730, Time=0.17 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2172.553, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2174.506, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2174.504, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2331.813, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=2175.404, Time=0.11 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 1.561 seconds


(0, 1, 1)

In [99]:
garch = arch_model(arima_residuals, p=1, q=1)
garch_fit = garch.fit() 


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1496.5180336534208
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1100.4835493621158
Iteration:      3,   Func. Count:     21,   Neg. LLF: 1086.012319142263
Iteration:      4,   Func. Count:     26,   Neg. LLF: 1085.6643651038073
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1080.9755880949115
Iteration:      6,   Func. Count:     36,   Neg. LLF: 302323689.0714041
Iteration:      7,   Func. Count:     46,   Neg. LLF: 1941.10123541118
Iteration:      8,   Func. Count:     53,   Neg. LLF: 1080.9254836060204
Iteration:      9,   Func. Count:     58,   Neg. LLF: 245488106.4451428
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1080.9254823951965
            Iterations: 11
            Function evaluations: 65
            Gradient evaluations: 9


In [100]:
predicted_mu = arima_model.predict(n_periods=1).item()
garch_forecast = garch_fit.forecast(horizon=1)
predicted_et = garch_forecast.mean['h.1'].iloc[-1]
prediction = predicted_mu + predicted_et
print(prediction)
predicted_price = train.iloc[-1] * (1 + prediction / 100)
print(f'Predicted price: {predicted_price}')
actual_price = test.iloc[0]
print(f'Actual price: {actual_price}')


0.3601806238512214
Predicted price: 436.62699937192735
Actual price: 450.04998779296875


C:\Users\surya\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\surya\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [68]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

def pacf(col):
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))
    ax1.plot(col)
    ax1.set_title('The Column Diff')
    plot_pacf(col, method='ywm', ax=ax2)
    

def acf(col):
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(15,5))
    ax1.plot(col)
    ax1.set_title('The Column Diff')
    plot_acf(col, ax=ax2)

In [69]:
arima_model.order

(0, 1, 1)